In [1]:
!ls -la

import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('./clothing-model.h5') #load using Keras
tf.saved_model.save(model, 'clothing-model-saved')  #save using Tensorflow

# RUN TF model in DOKCER:
# docker run -it --rm -p 8500:8500 -v "$(pwd)/clothing-model-saved:/model/clothing-model/1" -e MODEL_NAME="clothing-model" tensorflow/serving:2.7.0

!saved_model_cli show --dir clothing-model-saved --all

SyntaxError: invalid syntax (<ipython-input-1-cade30f67c77>, line 9)

In [9]:
# !pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0


     |████████████████████████████████| 4.0 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 489.6 MB 5.0 kB/s eta 0:00:018   |█                               | 14.7 MB 3.8 MB/s eta 0:02:07     |████▋                           | 71.4 MB 2.0 MB/s eta 0:03:29     |█████▉                          | 89.4 MB 3.5 MB/s eta 0:01:55     |██████████████████▍             | 280.9 MB 554 kB/s eta 0:06:17     |████████████████████            | 306.2 MB 78 kB/s eta 0:39:10
     |████████████████████████████████| 2.1 MB 69 kB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 386 kB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 76 kB/s  eta 0:00:01


  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [19]:
##create grpc client and load predict image

import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor


host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

url = 'http://bit.ly/mlbookcamp-pants'
preprocessor = create_preprocessor('xception', target_size=(299,299))
X = preprocessor.from_url(url)
X


array([[[[-0.11372548, -0.15294117, -0.19999999],
         [-0.11372548, -0.15294117, -0.19999999],
         [-0.10588235, -0.14509803, -0.19215685],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.10588235, -0.10588235, -0.16862744]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.09019607, -0.12941176, -0.17647058],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09019607, -0.09019607, -0.15294117]],

        ...,

        [[-0.67058825, -0.7019608 , -0

In [28]:
#call grpc predict

def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name="clothing-model"
pb_request.model_spec.signature_name = "serving_default"

# display(np_to_protobuf(X)) #request pants as array in protobuf
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

pb_response = stub.Predict(pb_request, timeout=20.0)
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8682903051376343
    float_val: -4.761245250701904
    float_val: -2.316983461380005
    float_val: -1.0625708103179932
    float_val: 9.887161254882812
    float_val: -2.8124334812164307
    float_val: -3.6662826538085938
    float_val: 3.200361728668213
    float_val: -2.6023378372192383
    float_val: -4.835046291351318
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [32]:
# parse response

preds = pb_response.outputs['dense_7'].float_val
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
dict(zip(classes, preds))

# To convert Jupyter notebook to script:
# !jupyter nbconvert --to script course10.ipynb

{'dress': -1.8682903051376343,
 'hat': -4.761245250701904,
 'longsleeve': -2.316983461380005,
 'outwear': -1.0625708103179932,
 'pants': 9.887161254882812,
 'shirt': -2.8124334812164307,
 'shoes': -3.6662826538085938,
 'shorts': 3.200361728668213,
 'skirt': -2.6023378372192383,
 't-shirt': -4.835046291351318}